In [8]:
is_notebook = get_ipython().has_trait("kernel")  # type: ignore

In [9]:
disk_map = input() or "2333133121414131402"

The disk map alternates between files and spaces, starting with files.


In [10]:
from dataclasses import dataclass
from typing import Union


@dataclass
class File:
    id: int


@dataclass
class Space:
    pass


DiskElement = Union[File, Space]

In [11]:
from typing import Iterable

In [12]:
def expand_disk_map(disk_map: str) -> Iterable[DiskElement]:
    for i, element_count in enumerate(disk_map):
        is_file = i % 2 == 0
        for _ in range(int(element_count)):
            if is_file:
                yield File(id=i // 2)
            else:
                yield Space()

In [13]:
def format_disk(disk: Iterable[DiskElement]) -> str:
    return "".join(
        f"{element.id}" if isinstance(element, File) else "." for element in disk
    )

In [14]:
format_disk(expand_disk_map("12345"))

'0..111....22222'

In [16]:
format_disk(expand_disk_map(disk_map)) if is_notebook else None

'00...111...2...333.44.5555.6666.777.888899'

In [18]:
def defrag_disk(
    disk: Iterable[DiskElement],
) -> Iterable[list[DiskElement]]:
    """Defrags the disk by repeatedly moving the last file to the leftmost available space."""
    result = list(disk)
    left = 0
    right = len(result) - 1

    yield result

    while left < right:
        if not isinstance(result[left], Space):
            left += 1
            continue
        if not isinstance(result[right], File):
            right -= 1
            continue

        result[left], result[right] = result[right], result[left]
        yield result

In [19]:
it = defrag_disk(expand_disk_map("12345"))

[format_disk(step) for step in it] if is_notebook else None

['0..111....22222',
 '02.111....2222.',
 '022111....222..',
 '0221112...22...',
 '02211122..2....',
 '022111222......']

In [21]:
it = defrag_disk(expand_disk_map(disk_map))

[format_disk(step) for step in it] if is_notebook else None

['00...111...2...333.44.5555.6666.777.888899',
 '009..111...2...333.44.5555.6666.777.88889.',
 '0099.111...2...333.44.5555.6666.777.8888..',
 '00998111...2...333.44.5555.6666.777.888...',
 '009981118..2...333.44.5555.6666.777.88....',
 '0099811188.2...333.44.5555.6666.777.8.....',
 '009981118882...333.44.5555.6666.777.......',
 '0099811188827..333.44.5555.6666.77........',
 '00998111888277.333.44.5555.6666.7.........',
 '009981118882777333.44.5555.6666...........',
 '009981118882777333644.5555.666............',
 '00998111888277733364465555.66.............',
 '0099811188827773336446555566..............']

In [22]:
*_, defragged = defrag_disk(list(expand_disk_map(disk_map)))

In [23]:
def checksum(disk: Iterable[DiskElement]) -> Iterable[int]:
    for i, x in enumerate(disk):
        if isinstance(x, File):
            yield i * x.id

In [24]:
print(sum(checksum(defragged)))

1928
